In [1]:
# ---------------------------------------------------------------------------
# 0️⃣  Setup – install / download once per runtime
# ---------------------------------------------------------------------------
import os, json, subprocess, sys
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump({"username": "dashanrichards", "key": "5f7a6f0b215f4b54682829ccf7c1fab4"}, f)
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

# ▸ Download + unzip dataset
subprocess.run(["kaggle", "datasets", "download",
                "-d", "psparks/instacart-market-basket-analysis", "--unzip"])


CompletedProcess(args=['kaggle', 'datasets', 'download', '-d', 'psparks/instacart-market-basket-analysis', '--unzip'], returncode=0)

In [2]:
# ---------------------------------------------------------------------------
# 1️⃣  Imports
# ---------------------------------------------------------------------------

import pandas as pd
import numpy as np
from typing import Tuple, List, Dict
from pathlib import Path


In [3]:
# ---------------------------------------------------------------------------
# 2️⃣  Modular pipeline functions
# ---------------------------------------------------------------------------

# ----- 2.1 Load raw data ----------------------------------------------------
def load_instacart_data(data_dir: Path = Path(".")) -> Tuple[pd.DataFrame, ...]:
    """Load Instacart CSVs from `data_dir`."""
    orders       = pd.read_csv(data_dir / "orders.csv")
    order_prior  = pd.read_csv(data_dir / "order_products__prior.csv")
    products     = pd.read_csv(data_dir / "products.csv")
    aisles       = pd.read_csv(data_dir / "aisles.csv")
    departments  = pd.read_csv(data_dir / "departments.csv")
    return orders, order_prior, products, aisles, departments

In [4]:
# ----- 2.2 Build timeline & daily demand ------------------------------------
def generate_order_timeline(
    orders: pd.DataFrame,
    order_prior: pd.DataFrame,
    epoch: str = "2023-01-01"
) -> pd.DataFrame:
    """Merge order header + line items and add pseudo-dates."""
    orders = orders[orders.eval_set == "prior"]
    merged = order_prior.merge(
        orders[["order_id", "user_id", "order_number",
                "order_dow", "order_hour_of_day", "days_since_prior_order"]],
        on="order_id", how="left"
    )
    merged["days_since_prior_order"] = merged["days_since_prior_order"].fillna(0)
    merged["order_date"] = (
        pd.to_timedelta(
            merged.groupby("user_id")["days_since_prior_order"].cumsum(), unit="D"
        ) + pd.to_datetime(epoch)
    )
    # Optional time features
    merged["dow"]        = merged["order_date"].dt.dayofweek
    merged["is_weekend"] = merged["dow"].isin([5, 6]).astype(int)
    return merged


def calculate_daily_demand(timeline_df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate to daily SKU demand."""
    return (
        timeline_df.groupby(["order_date", "product_id"])
        .size()
        .reset_index(name="demand")
        .rename(columns={"order_date": "date"})
    )

In [5]:
# ----- 2.3  Inventory config -------------------------------------------------
def setup_inventory(
    product_ids: np.ndarray,
    start_date: pd.Timestamp,
    end_date: pd.Timestamp,
    products_meta: pd.DataFrame | None = None,
) -> Tuple[pd.DataFrame, pd.DatetimeIndex]:
    """Create random inventory policies (editable later by agents)."""
    inv_cfg = pd.DataFrame({
        "product_id": product_ids,
        "initial_inventory"    : np.random.randint(500, 1000, size=len(product_ids)),
        "restock_every_n_days" : np.random.choice([7, 14],  size=len(product_ids)),
        "restock_amount"       : np.random.randint(300, 600,  size=len(product_ids)),
    }).set_index("product_id")

    if products_meta is not None:
        products_meta = products_meta.reset_index()
        inv_cfg = (
            inv_cfg.merge(products_meta, on="product_id", how="left")
                   .set_index("product_id")
        )
    sim_days = pd.date_range(start=start_date, end=end_date, freq="D")
    return inv_cfg, sim_days

In [6]:
# ----- 2.4  Core simulation --------------------------------------------------
def simulate_stockouts(
    daily_demand: pd.DataFrame,
    inv_cfg: pd.DataFrame,
    sim_days: pd.DatetimeIndex,
) -> pd.DataFrame:
    """
    Loop-based simulation (easy to read & agent-friendly).
    Vectorization is possible later if scale becomes an issue.
    """
    # State dict for every SKU
    state: Dict[int, Dict[str, any]] = {
        pid: {"inv": inv_cfg.loc[pid, "initial_inventory"],
              "last_restock": sim_days[0]}
        for pid in inv_cfg.index
    }

    records: List[Dict] = []

    demand_grouped = {d: df for d, df in daily_demand.groupby("date", sort=False)}

    for day in sim_days:
        daily_df = demand_grouped.get(day, None)
        if daily_df is None:
            continue  # no demand that day

        for _, row in daily_df.iterrows():
            pid, demand = int(row.product_id), int(row.demand)
            cfg   = inv_cfg.loc[pid]
            s     = state[pid]

            # Restock if due
            if (day - s["last_restock"]).days >= cfg.restock_every_n_days:
                s["inv"]         += cfg.restock_amount
                s["last_restock"] = day

            fulfilled = min(demand, s["inv"])
            stockout  = int(fulfilled < demand)
            s["inv"] -= fulfilled

            records.append({
                "date"               : day,
                "product_id"         : pid,
                "demand"             : demand,
                "fulfilled"          : fulfilled,
                "stockout"           : stockout,
                "remaining_inventory": s["inv"],
            })

    return pd.DataFrame(records)


In [7]:
# ---------------------------------------------------------------------------
# 3️⃣  Driver / orchestrator (can be run as `python stockout_sim.py`)
# ---------------------------------------------------------------------------

if __name__ == "__main__":
    print("🔄 Loading data …")
    ORDERS, PRIOR, PRODUCTS, AISLES, DEPTS = load_instacart_data()

    print("📅 Building timeline …")
    order_timeline      = generate_order_timeline(ORDERS, PRIOR)
    daily_demand_df     = calculate_daily_demand(order_timeline)

    print("🏗️  Preparing inventory config …")
    product_meta = (
        PRODUCTS.merge(AISLES, on="aisle_id", how="left")
                .merge(DEPTS, on="department_id", how="left")
                .set_index("product_id")
    )

    INV_CFG, SIM_DAYS = setup_inventory(
        daily_demand_df.product_id.unique(),
        daily_demand_df.date.min(),
        daily_demand_df.date.max(),
        products_meta=product_meta,
    )

    print("🚚 Simulating inventory flow …")
    stockout_df = simulate_stockouts(daily_demand_df, INV_CFG, SIM_DAYS)

🔄 Loading data …
📅 Building timeline …
🏗️  Preparing inventory config …
🚚 Simulating inventory flow …


In [8]:
    # -----------------------------------------------------------------------
    # 4️⃣  Output / hand-off for agents, dashboards, ML, etc.
    # -----------------------------------------------------------------------

    print("\n✅ Simulation complete.")
    print(stockout_df.head())

    # Example KPI
    kpi = stockout_df.stockout.mean()
    print(f"\n🔎  Overall stockout rate: {kpi:.2%}")

    # Save for later steps (forecasting, LLM prompts, dashboards)
    stockout_df.to_csv("stockout_results.csv", index=False)


✅ Simulation complete.
        date  product_id  demand  fulfilled  stockout  remaining_inventory
0 2023-01-01           1      15         15         0                  519
1 2023-01-01           3       2          2         0                  575
2 2023-01-01           4       7          7         0                  868
3 2023-01-01           7       1          1         0                  654
4 2023-01-01           8       2          2         0                  706

🔎  Overall stockout rate: 0.06%


In [9]:
# Top SKUs by stockout frequency
sku_stockout_summary = stockout_df.groupby('product_id').agg(
    total_days=('date', 'count'),
    stockout_days=('stockout', 'sum'),
    avg_remaining_inv=('remaining_inventory', 'mean'),
    total_demand=('demand', 'sum')
).reset_index()

sku_stockout_summary['stockout_rate'] = (
    sku_stockout_summary['stockout_days'] / sku_stockout_summary['total_days']
)

# Sort by worst stockout rate
top_stockout_skus = sku_stockout_summary.sort_values('stockout_rate', ascending=False).head(10)
print(top_stockout_skus)

       product_id  total_days  stockout_days  avg_remaining_inv  total_demand  \
24848       24852        9534           2057       52503.506503        472565   
26204       26209        8172            887       37603.587004        140627   
47755       47766        8254            816       53490.816331        176815   
27839       27845        7925            781       37372.263470        137905   
21133       21137        9307            874       89304.953691        264683   
13172       13176        9325            796      160184.040107        379450   
47615       47626        8157            662       56272.565649        152657   
21899       21903        8875            554      125959.421859        241921   
16793       16797        7887            396       91972.548878        142951   
4917         4920        7576            366       43478.517423         82689   

       stockout_rate  
24848       0.215754  
26204       0.108541  
47755       0.098861  
27839       0.09

In [11]:
!pip install shap
!pip install lightgbm
!pip install xgboost
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.9 MB/s eta 0:00:00


In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import shap
import time
import joblib
import warnings

In [13]:
warnings.filterwarnings("ignore")

# --- Load preprocessed dataset ---
stockout_df = pd.read_csv("stockout_results.csv")

In [14]:
# --- Feature Engineering ---
stockout_df['day_of_week'] = pd.to_datetime(stockout_df['date']).dt.dayofweek
stockout_df['demand_lag_1'] = stockout_df.groupby('product_id')['demand'].shift(1)
stockout_df['inventory_lag_1'] = stockout_df.groupby('product_id')['remaining_inventory'].shift(1)
stockout_df['target'] = stockout_df.groupby('product_id')['stockout'].shift(-1)

model_df = stockout_df.dropna(subset=['demand_lag_1', 'inventory_lag_1', 'target'])

X = model_df[['day_of_week', 'demand', 'demand_lag_1', 'inventory_lag_1']]
y = model_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Define Models ---
models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest"      : RandomForestClassifier(n_estimators=100, random_state=42),
    "LightGBM"           : LGBMClassifier(verbose=-1),
    "XGBoost"            : XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0),
    "CatBoost"           : CatBoostClassifier(verbose=0)
}


In [15]:
# --- Benchmark Models ---
results = []
trained_models = {}

print(f"{'Model':<22} | {'F1':>5} | {'Precision':>9} | {'Recall':>7} | {'Train Time (s)':>14}")
print("-" * 70)

for name, model in models.items():
    start = time.time()
    model.fit(X_train, y_train)
    end = time.time()

    preds = model.predict(X_test)
    f1 = f1_score(y_test, preds)
    prec = precision_score(y_test, preds)
    rec = recall_score(y_test, preds)
    duration = end - start

    results.append({
        "Model": name,
        "F1 Score": round(f1, 3),
        "Precision": round(prec, 3),
        "Recall": round(rec, 3),
        "Train Time (s)": round(duration, 3)
    })

    trained_models[name] = model
    print(f"{name:<22} | {f1:>5.3f} | {prec:>9.3f} | {rec:>7.3f} | {duration:>14.3f}")

Model                  |    F1 | Precision |  Recall | Train Time (s)
----------------------------------------------------------------------
Logistic Regression    | 0.653 |     0.811 |   0.546 |         13.417
Random Forest          | 0.911 |     0.941 |   0.883 |       1112.649
LightGBM               | 0.341 |     0.514 |   0.255 |          8.452
XGBoost                | 0.504 |     0.489 |   0.521 |          7.610
CatBoost               | 0.595 |     0.706 |   0.514 |        168.502


In [16]:
# --- Results DataFrame ---
results_df = pd.DataFrame(results).sort_values(by='F1 Score', ascending=False).reset_index(drop=True)

In [17]:
# --- SHAP Explainability for Best Model ---
best_model_name = results_df.iloc[0]["Model"]
best_model = trained_models[best_model_name]

print(f"\n🔍 Best Model: {best_model_name}")

X_train_np = X_train.to_numpy()


🔍 Best Model: Random Forest


In [18]:
# --- Save the Best Model ---
joblib.dump(best_model, f"best_model_{best_mvodel_name.replace(' ', '_').lower()}.pkl")
print(f"✅ Model saved to: best_model_{best_model_name.replace(' ', '_').lower()}.pkl")

✅ Model saved to: best_model_random_forest.pkl
